<a href="https://colab.research.google.com/github/caioitalo/soulcode/blob/main/Pr%C3%A1tica_Pyspark_Spotify_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Prática realizada em pyspark para tratamento de dados de uma DB com músicas do spotify com aproximadamente 110 mil linhas.**

DB retirado da plataforma kaggle - [link](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset) e alocado em um bucket da GCP

# EXTRAÇÃO DOS DADOS

In [1]:
!pip install pyspark
!pip install gcsfs # instalação da biblioteca da GC Storage
!pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3b0cc2229ed29f1aa03095b10d3523b6c0f43c4da597408a86f75e70330f141d
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 KB 5.0 MB/s eta 0:00:00


In [2]:
#importação da biblioteca da google cloud storage e o sistema operacional
import pandas as pd
from pyspark.sql import SparkSession
from google.cloud import storage
import os
import pyspark.sql.functions as F
import numpy as np
import pandera as pa
from pyspark.sql.types import *

In [3]:
#configuração da conta
serviceAccount = '/content/sc-bc26-ed7-adb0dc2607d9.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [4]:
# criando o acesso e definindo uma variável para representá-lo
client = storage.Client()
bucket = client.get_bucket('datalake-soulcode-ed7')

#acessando o arquivo
bucket.blob('arquivos soulcode/spotify.csv')

#criando uma variável para acesar o caminho, via gsutil URI que está nos detalhes do arquivo
path = 'gs://datalake-soulcode-ed7/arquivos soulcode/spotify.csv'

In [5]:
# essa spark.jars é padrão para quando fizer isso com o storage
spark = (
    SparkSession.builder
                .master('local')
                .appName('gcsfs')
                .config('spark.ui.port', '4050')
                .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                .getOrCreate()
)

In [6]:
esquema = (
          StructType([
          StructField('Unnamed: 0', IntegerType()),
	        StructField('track_id', StringType()),
          StructField('artists', StringType()),
          StructField('album_name', StringType()),
          StructField('track_name', StringType()),
          StructField('popularity', IntegerType(), True),
          StructField('duration_ms', StringType(), True),
          StructField('explicit', StringType(),True),
          StructField('danceability', FloatType(),True),
          StructField('energy', FloatType(),True),
          StructField('key', StringType(),True),
          StructField('loudness', FloatType(),True),
          StructField('mode', StringType(),True),
          StructField('speechiness', StringType(),True),
          StructField('acousticness',StringType(),True),
          StructField('instrumentalness', StringType(),True),
          StructField('liveness', StringType(),True),
          StructField('valence', StringType(),True),
          StructField('tempo', FloatType(),True),
          StructField('time_signature', IntegerType(),True),
          StructField('track_genre', StringType())
          ])
)
          

In [7]:
df_pand = pd.read_csv(path)

In [8]:
df = spark.createDataFrame(df_pand,schema=esquema)

In [9]:
df.show()

+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|Unnamed: 0|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|         0|5SuOikwiRyPMVoIQD...|         Gen Hoshino|              Comedy|              Comedy|        73|     230666|   false|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|         1.01E-6|   0.358|

# TRATAMENTOS DOS DADOS

ATIVIDADE SPARK - SPOTIFY

1. RENOMEAR TODAS AS COLUNAS (TRADUZINDO PARA O PORTUGUÊS) 
2. FAÇA UMA ANÁLISE INICIAL, ORDENANDO POR COLUNAS DE FORMA ASCENDENTE E DESCENDENTE 
3. VERIFIQUE SE HÁ ALGUMA COLUNA QUE NÃO TENHA INFORMAÇÃO RELEVANTE E FAÇA UM DROP 
4. VERIFIQUE SE HÁ LINHAS DUPLICADAS (DUPLICATAS) E REMOVA-AS. 
5. VERIFIQUE SE HÁ DADOS INCONSISTENTES 
6. REMOVA AS INCONSISTÊNCIAS 
7. VERIFIQUE OS TIPOS DE DADOS DAS COLUNAS, E COLOQUE PARA UM TIPO MAIS ADEQUADO. 
8. FAÇA OUTRAS VERIFICAÇÕES DE ACORDO COM A SUA EXPERIÊNCIA EM PANDAS. 
9. ENCONTRE ALGUMAS ANÁLISES INTERESSANTES UTILIZANDO AS FUNÇÕES DO SPARK VISTAS EM AULA E LISTE PELO MENOS 5 DESSAS.

In [10]:
# fazendo um backup do Dataframe inicial
dfbackup = df

In [11]:
df = dfbackup

## RENOMEAÇÃO DE TODAS AS COLUNAS

In [12]:
df.dtypes

[('Unnamed: 0', 'int'),
 ('track_id', 'string'),
 ('artists', 'string'),
 ('album_name', 'string'),
 ('track_name', 'string'),
 ('popularity', 'int'),
 ('duration_ms', 'string'),
 ('explicit', 'string'),
 ('danceability', 'float'),
 ('energy', 'float'),
 ('key', 'string'),
 ('loudness', 'float'),
 ('mode', 'string'),
 ('speechiness', 'string'),
 ('acousticness', 'string'),
 ('instrumentalness', 'string'),
 ('liveness', 'string'),
 ('valence', 'string'),
 ('tempo', 'float'),
 ('time_signature', 'int'),
 ('track_genre', 'string')]

In [13]:
df = df.drop('track_id', 'time_signature')

In [14]:
df = (df.withColumnRenamed('Unnamed: 0','id_musica')
        .withColumnRenamed('track_name','nome')
        .withColumnRenamed('popularity','popularidade')
        .withColumnRenamed('duration_ms','duracao_ms')
        .withColumnRenamed('acousticness','nivel_acustico')
        .withColumnRenamed('danceability','dancabilidade')
        .withColumnRenamed('explicit','cont_explic')
        .withColumnRenamed('energy','energia')
        .withColumnRenamed('instrumentalness','nivel_instrumental')
        .withColumnRenamed('key','tom_musical')
        .withColumnRenamed('liveness','nivel_aovivo')
        .withColumnRenamed('loudness','volume_db')
        .withColumnRenamed('mode','esc_melodica')
        .withColumnRenamed('speechiness','nivel_fala')
        .withColumnRenamed('valence','positividade')
        .withColumnRenamed('tempo','compasso')
        .withColumnRenamed('artists','artistas')
        .withColumnRenamed('album_name','album')
        .withColumnRenamed('track_genre','genero')
)


In [15]:
df.show(2)

+---------+------------+----------------+----------------+------------+----------+-----------+-------------+-------+-----------+---------+------------+----------+--------------+------------------+------------+------------+--------+--------+
|id_musica|    artistas|           album|            nome|popularidade|duracao_ms|cont_explic|dancabilidade|energia|tom_musical|volume_db|esc_melodica|nivel_fala|nivel_acustico|nivel_instrumental|nivel_aovivo|positividade|compasso|  genero|
+---------+------------+----------------+----------------+------------+----------+-----------+-------------+-------+-----------+---------+------------+----------+--------------+------------------+------------+------------+--------+--------+
|        0| Gen Hoshino|          Comedy|          Comedy|          73|    230666|      false|        0.676|  0.461|          1|   -6.746|           0|     0.143|        0.0322|           1.01E-6|       0.358|       0.715|  87.917|acoustic|
|        1|Ben Woodward|Ghost (Acous

##ANÁLISE INICIAL, ORDENANDO POR COLUNAS DE FORMA ASCENDENTE E DESCENDENTE

In [16]:
df.select('*').orderBy('nome').show(5)

+---------+--------------------+--------------------+--------------------+------------+----------+-----------+-------------+-------+-----------+---------+------------+----------+--------------+------------------+------------+------------+--------+----------+
|id_musica|            artistas|               album|                nome|popularidade|duracao_ms|cont_explic|dancabilidade|energia|tom_musical|volume_db|esc_melodica|nivel_fala|nivel_acustico|nivel_instrumental|nivel_aovivo|positividade|compasso|    genero|
+---------+--------------------+--------------------+--------------------+------------+----------+-----------+-------------+-------+-----------+---------+------------+----------+--------------+------------------+------------+------------+--------+----------+
|    36750|               Rilès|      !I'll Be Back!|      !I'll Be Back!|          52|    178533|       true|        0.823|  0.612|          1|   -7.767|           1|     0.248|         0.168|               0.0|       0.10

In [17]:
df.select('*').orderBy(F.col('popularidade').desc()).show(5)

+---------+--------------------+--------------------+--------------------+------------+----------+-----------+-------------+-------+-----------+---------+------------+----------+--------------+------------------+------------+------------+--------+-------+
|id_musica|            artistas|               album|                nome|popularidade|duracao_ms|cont_explic|dancabilidade|energia|tom_musical|volume_db|esc_melodica|nivel_fala|nivel_acustico|nivel_instrumental|nivel_aovivo|positividade|compasso| genero|
+---------+--------------------+--------------------+--------------------+------------+----------+-----------+-------------+-------+-----------+---------+------------+----------+--------------+------------------+------------+------------+--------+-------+
|    81051|Sam Smith;Kim Petras|Unholy (feat. Kim...|Unholy (feat. Kim...|         100|    156943|      false|        0.714|  0.472|          2|   -7.375|           1|    0.0864|         0.013|           4.51E-6|       0.266|       

In [18]:
# Reorganização de colunas
df = df.select(['id_musica','nome','artistas','album','duracao_ms','popularidade','cont_explic','tom_musical','esc_melodica',
                'dancabilidade','energia','volume_db','nivel_fala','nivel_acustico','nivel_instrumental','nivel_aovivo','positividade',
                'compasso','genero'])

## VERIFICAÇÃO DE LINHAS DUPLICADAS (DUPLICATAS)

In [19]:
# Verificando o número de linhas antes da verificação de duplicidades
df.count()

114000

In [20]:
# Vê-se que há músicas repetidas
dfcont = df.groupBy('nome').count()
dfcont.sort(dfcont['count'].desc()).show(truncate=False)

+---------------------------------+-----+
|nome                             |count|
+---------------------------------+-----+
|Run Rudolph Run                  |151  |
|Halloween                        |88   |
|Frosty The Snowman               |81   |
|Little Saint Nick - 1991 Remix   |76   |
|Last Last                        |75   |
|Christmas Time                   |72   |
|CÓMO SE SIENTE - Remix           |64   |
|Sleigh Ride                      |61   |
|RUMBATÓN                         |60   |
|X ÚLTIMA VEZ                     |58   |
|Ley Seca                         |54   |
|Feliz Cumpleaños Ferxxo          |54   |
|Rudolph The Red-Nosed Reindeer   |52   |
|Christmas All Over Again         |52   |
|On Repeat                        |52   |
|Qué Más Pues?                    |51   |
|Hot in It                        |51   |
|Christmas Without You            |51   |
|Rockin' Around The Christmas Tree|49   |
|Secreto                          |47   |
+---------------------------------

In [21]:
# usando a música 'Halloween' como exemplo vê-se que há duplicidades. O que muda é o ID, será dropado e adicionado quando for manipulado com pandas.
# também nota-se que há músicas iguais mudando-se apenas o nome do álbum e gênero
df.filter(df.nome == 'Halloween').show(20)

+---------+---------+--------------------+--------------------+----------+------------+-----------+-----------+------------+-------------+-------+---------+----------+--------------+------------------+------------+------------+--------+-----------+
|id_musica|     nome|            artistas|               album|duracao_ms|popularidade|cont_explic|tom_musical|esc_melodica|dancabilidade|energia|volume_db|nivel_fala|nivel_acustico|nivel_instrumental|nivel_aovivo|positividade|compasso|     genero|
+---------+---------+--------------------+--------------------+----------+------------+-----------+-----------+------------+-------------+-------+---------+----------+--------------+------------------+------------+------------+--------+-----------+
|     2783|Halloween|Siouxsie and the ...|Halloween Scary P...|    221466|           0|      false|          7|           1|        0.223|   0.91|   -7.097|      0.08|        0.0116|           5.18E-4|       0.212|       0.364|  171.75|   alt-rock|
|   

In [22]:
df = df.drop('id_musica')
df = df.dropDuplicates()

In [23]:
# contando as linhas após o drop de duplicidades. Note que saiu de 114 mil para 113,4 mil.
df.count()

113423

In [24]:
# Se considerar as possíveis inconsistências por conta do gênero e diferentes álbums, pode-se notar que o número de músicas vai de 114k para 86,7k
df2 = df
df2 = df2.drop('genero','album')
df2 = df2.dropDuplicates()
df2.count()

86736

## TRATAMENTO DOS DADOS INCONSISTENTES

In [25]:
# checando valores nulos nas colunas
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+--------+-----+----------+------------+-----------+-----------+------------+-------------+-------+---------+----------+--------------+------------------+------------+------------+--------+------+
|nome|artistas|album|duracao_ms|popularidade|cont_explic|tom_musical|esc_melodica|dancabilidade|energia|volume_db|nivel_fala|nivel_acustico|nivel_instrumental|nivel_aovivo|positividade|compasso|genero|
+----+--------+-----+----------+------------+-----------+-----------+------------+-------------+-------+---------+----------+--------------+------------------+------------+------------+--------+------+
|   1|       1|    1|         0|           0|          0|          0|           0|            0|      0|        0|         0|             0|                 0|           0|           0|       0|     0|
+----+--------+-----+----------+------------+-----------+-----------+------------+-------------+-------+---------+----------+--------------+------------------+------------+------------+-------

In [26]:
# excluindo a linha com valor nulo
df = df.where(F.col('nome') != 'NaN')

In [27]:
df.dtypes

[('nome', 'string'),
 ('artistas', 'string'),
 ('album', 'string'),
 ('duracao_ms', 'string'),
 ('popularidade', 'int'),
 ('cont_explic', 'string'),
 ('tom_musical', 'string'),
 ('esc_melodica', 'string'),
 ('dancabilidade', 'float'),
 ('energia', 'float'),
 ('volume_db', 'float'),
 ('nivel_fala', 'string'),
 ('nivel_acustico', 'string'),
 ('nivel_instrumental', 'string'),
 ('nivel_aovivo', 'string'),
 ('positividade', 'string'),
 ('compasso', 'float'),
 ('genero', 'string')]

In [28]:
# checando cada coluna a procura de inconsistências
df.select('genero').distinct().orderBy('genero').show(100,truncate=False)

+-----------------+
|genero           |
+-----------------+
|acoustic         |
|afrobeat         |
|alt-rock         |
|alternative      |
|ambient          |
|anime            |
|black-metal      |
|bluegrass        |
|blues            |
|brazil           |
|breakbeat        |
|british          |
|cantopop         |
|chicago-house    |
|children         |
|chill            |
|classical        |
|club             |
|comedy           |
|country          |
|dance            |
|dancehall        |
|death-metal      |
|deep-house       |
|detroit-techno   |
|disco            |
|disney           |
|drum-and-bass    |
|dub              |
|dubstep          |
|edm              |
|electro          |
|electronic       |
|emo              |
|folk             |
|forro            |
|french           |
|funk             |
|garage           |
|german           |
|gospel           |
|goth             |
|grindcore        |
|groove           |
|grunge           |
|guitar           |
|happy            |


Inconsistências iniciais encontradas: 
1. duracao_ms que deve ser int - check
2. transformar cont_explic para boolean - check
3. transformar tom_musical para os tons de fato, maior_ou_menor para maior ou menor - check
4. transformar tipos de outras colunas - check 


In [29]:
# troca de colunas para um tipo mais condizente com os dados
df = (df.withColumn('duracao_ms',F.col('duracao_ms').cast(IntegerType()))
        .withColumn('cont_explic',F.col('cont_explic').cast(BooleanType()))
        .withColumn('nivel_fala',F.col('nivel_fala').cast(FloatType()))
        .withColumn('nivel_acustico',F.col('nivel_acustico').cast(FloatType()))
        .withColumn('nivel_instrumental',F.col('nivel_instrumental').cast(FloatType()))
        .withColumn('nivel_aovivo',F.col('nivel_aovivo').cast(FloatType()))
        .withColumn('positividade',F.col('positividade').cast(FloatType()))
)

In [30]:
df.dtypes

[('nome', 'string'),
 ('artistas', 'string'),
 ('album', 'string'),
 ('duracao_ms', 'int'),
 ('popularidade', 'int'),
 ('cont_explic', 'boolean'),
 ('tom_musical', 'string'),
 ('esc_melodica', 'string'),
 ('dancabilidade', 'float'),
 ('energia', 'float'),
 ('volume_db', 'float'),
 ('nivel_fala', 'float'),
 ('nivel_acustico', 'float'),
 ('nivel_instrumental', 'float'),
 ('nivel_aovivo', 'float'),
 ('positividade', 'float'),
 ('compasso', 'float'),
 ('genero', 'string')]

In [ ]:
# utilizando o pandas para criar um novo ID para as músicas e retornando para o pyspark
dfpand = df.toPandas()
dfpand.insert(0,'id_musica', 0)

for i in range(len(dfpand)):
  dfpand['id_musica'][i] = int(i)

df = spark.createDataFrame(dfpand)

In [33]:
df.show()

+---------+--------------------+--------------------+--------------------+----------+------------+-----------+-----------+------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+------------------+-----------+
|id_musica|                nome|            artistas|               album|duracao_ms|popularidade|cont_explic|tom_musical|esc_melodica|      dancabilidade|            energia|          volume_db|          nivel_fala|      nivel_acustico|  nivel_instrumental|       nivel_aovivo|       positividade|          compasso|     genero|
+---------+--------------------+--------------------+--------------------+----------+------------+-----------+-----------+------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+------------------+-----------+
|        0

In [34]:
# Conferência se os tipos estão de acordo com cada coluna
df.dtypes

[('id_musica', 'bigint'),
 ('nome', 'string'),
 ('artistas', 'string'),
 ('album', 'string'),
 ('duracao_ms', 'bigint'),
 ('popularidade', 'bigint'),
 ('cont_explic', 'boolean'),
 ('tom_musical', 'string'),
 ('esc_melodica', 'string'),
 ('dancabilidade', 'double'),
 ('energia', 'double'),
 ('volume_db', 'double'),
 ('nivel_fala', 'double'),
 ('nivel_acustico', 'double'),
 ('nivel_instrumental', 'double'),
 ('nivel_aovivo', 'double'),
 ('positividade', 'double'),
 ('compasso', 'double'),
 ('genero', 'string')]

In [35]:
# Ajuste dos valores nas linhas de escala melódica e tom musical para algo mais compreensível
df = df.withColumn('esc_melodica',(F.when(df['esc_melodica'] == 0, 'Menor').otherwise('Maior')))

In [36]:
df = df.withColumn("tom_musical", F.when(df.tom_musical == 0,"Dó")
                                   .when(df.tom_musical == 1,"Ré Bemol")
                                   .when(df.tom_musical == 2,"Ré")
                                   .when(df.tom_musical == 3,"Mi Bemol")
                                   .when(df.tom_musical == 4,"Mi")
                                   .when(df.tom_musical == 5,"Fá")
                                   .when(df.tom_musical == 6,"Sol Bemol")
                                   .when(df.tom_musical == 7,"Sol")
                                   .when(df.tom_musical == 8,"Lá Bemol")
                                   .when(df.tom_musical == 9,"Lá")
                                   .when(df.tom_musical == 10,"Si Bemol")
                                   .otherwise("Si"))

In [38]:
df.show(2,truncate=False)

+---------+----------------+-------------+------------------+----------+------------+-----------+-----------+------------+------------------+------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+------------------+--------+
|id_musica|nome            |artistas     |album             |duracao_ms|popularidade|cont_explic|tom_musical|esc_melodica|dancabilidade     |energia           |volume_db          |nivel_fala          |nivel_acustico     |nivel_instrumental  |nivel_aovivo       |positividade       |compasso          |genero  |
+---------+----------------+-------------+------------------+----------+------------+-----------+-----------+------------+------------------+------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+------------------+--------+
|0        |Rain - Long Ver.|Motohiro Hata|言ノ葉            |447306   

## **Validação de dados utilizando Pandera**

In [41]:
df.dtypes

[('id_musica', 'bigint'),
 ('nome', 'string'),
 ('artistas', 'string'),
 ('album', 'string'),
 ('duracao_ms', 'bigint'),
 ('popularidade', 'bigint'),
 ('cont_explic', 'boolean'),
 ('tom_musical', 'string'),
 ('esc_melodica', 'string'),
 ('dancabilidade', 'double'),
 ('energia', 'double'),
 ('volume_db', 'double'),
 ('nivel_fala', 'double'),
 ('nivel_acustico', 'double'),
 ('nivel_instrumental', 'double'),
 ('nivel_aovivo', 'double'),
 ('positividade', 'double'),
 ('compasso', 'double'),
 ('genero', 'string')]

In [68]:
# utilizando pandera para validação dos dados com definição de checks para intervalos de valores.
schema = pa.DataFrameSchema({
    "id_musica": pa.Column(int),
    "nome": pa.Column(str),
    'artistas':pa.Column(str),
    'album':pa.Column(str),
    'duracao_ms':pa.Column(int, pa.Check.greater_than_or_equal_to(0)),
    'popularidade':pa.Column(int, pa.Check.in_range(0,100)),
    'cont_explic':pa.Column(bool),
    'tom_musical':pa.Column(str),
    'esc_melodica':pa.Column(str),
    'dancabilidade':pa.Column(float, pa.Check.in_range(0,1)),
    'energia':pa.Column(float, pa.Check.in_range(0,1)),
    'volume_db':pa.Column(float),
    'nivel_fala':pa.Column(float, pa.Check.in_range(0,1)),
    'nivel_acustico':pa.Column(float, pa.Check.in_range(0,1)),
    'nivel_instrumental':pa.Column(float, pa.Check.in_range(0,1)),
    'nivel_aovivo':pa.Column(float, pa.Check.in_range(0,1)),
    'positividade':pa.Column(float, pa.Check.in_range(0,1)),
    'compasso':pa.Column(float, pa.Check.greater_than_or_equal_to(0)),
    'genero':pa.Column(str)
})

schema.validate(df.toPandas())

,id_musica,nome,artistas,album,duracao_ms,popularidade,cont_explic,tom_musical,esc_melodica,dancabilidade,energia,volume_db,nivel_fala,nivel_acustico,nivel_instrumental,nivel_aovivo,positividade,compasso,genero
0,0,Rain - Long Ver.,Motohiro Hata,言ノ葉,447306,44,False,Lá,Maior,0.588,0.604,-7.602,0.0240,0.343000,0.000069,0.2520,0.3510,92.019997,acoustic
1,1,Psycho,Harley Poe,Have a Great Life.,241250,30,False,Sol Bemol,Menor,0.458,0.823,-4.268,0.0616,0.382000,0.000000,0.2050,0.4220,131.638000,acoustic
2,2,Dancing With A Stranger,Sara Farell,Dancing With A Stranger,188735,43,False,Lá,Maior,0.595,0.180,-11.391,0.0298,0.890000,0.000000,0.1130,0.2610,79.921997,acoustic
3,3,To the Wind,Chris Cresswell,To the Wind,187908,25,False,Si,Maior,0.491,0.561,-7.225,0.0246,0.108000,0.000000,0.1390,0.2200,151.942993,acoustic
4,4,Tô Pra Ver,Criolo;Rael,Ainda Há Tempo,254520,41,False,Lá Bemol,Maior,0.637,0.818,-6.851,0.2810,0.190000,0.000000,0.1020,0.5990,90.570999,afrobeat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113417,113417,Imaginarium - Mix Edit,NG Rezonance;PHD,Hard Trance Workout,342857,29,False,Sol,Menor,0.478,0.947,-7.206,0.0350,0.000311,0.938000,0.1790,0.1790,139.968994,trance
113418,113418,Bullets,Archive,Controlling Crowds (Parts I-III),353573,52,False,Lá Bemol,Maior,0.455,0.649,-10.444,0.0519,0.126000,0.000353,0.1550,0.0719,164.994003,trip-hop
113419,113419,Yolu Yok (feat. Zerrin),Serhat Durmus;Zerrin,Yolu Yok (feat. Zerrin),195991,44,False,Lá,Menor,0.655,0.632,-8.655,0.0454,0.067700,0.000599,0.0907,0.1300,89.985001,turkish
113420,113420,Yaşıyorum,Candan Erçetin,Melek,252400,38,False,Sol,Menor,0.442,0.693,-5.126,0.0908,0.330000,0.000000,0.1740,0.6990,96.397003,turkish


## SPARK SQL

**Utilização de Spark SQL para realização de queries e observação de informações relevantes com o dataframe já tratado**

In [72]:
# Conversão do dataframe com os dados tratados para o formato de tabela a fim de que se use SQL 
df.write.saveAsTable('spotify')

In [75]:
# Agora é possível utilizar o spark SQL 
spark.sql('SHOW TABLES').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|  spotify|      false|
+---------+---------+-----------+



In [82]:
# QUERY 1 - Ordenação das 10 maiores músicas em minutos
spark.sql('''SELECT nome, artistas, duracao_ms/60000 AS duracao_min
             FROM spotify
             ORDER BY duracao_ms DESC''').show(10,truncate=False)

+--------------------------------------------+-----------------+-----------------+
|nome                                        |artistas         |duracao_min      |
+--------------------------------------------+-----------------+-----------------+
|Unity (Voyage Mix) Pt. 1                    |Tale Of Us       |87.28825         |
|Crossing Wires 002 - Continuous DJ Mix      |Timo Maas        |79.8171          |
|The Lab 03 - Continuous DJ Mix Part 1       |Seth Troxler     |78.83836666666667|
|Amnesia Ibiza Underground 10 DJ Mix         |Loco Dice        |76.06495         |
|House of Om - Mark Farina - Continuous Mix  |Mark Farina      |74.12533333333333|
|Live In Tokyo - Continuous Mix              |Mark Farina      |72.33043333333333|
|Greenhouse Construction                     |Mark Farina      |72.24535         |
|NQ State of Mind, Vol. 1 - Continuous DJ Mix|Lenzman;Dan Stezo|70.7701          |
|Ocean Waves Sounds                          |Ocean Sounds     |68.67096666666667|
|Int

In [103]:
# QUERY 2 - Quais tons são mais comuns nas músicas do dataset?
spark.sql('''SELECT tom_musical, COUNT(tom_musical) as num_musicas
             FROM spotify
             GROUP BY tom_musical
             ORDER BY num_musicas DESC''').show(15,truncate=False)

+-----------+-----------+
|tom_musical|num_musicas|
+-----------+-----------+
|Sol        |13186      |
|Dó         |12983      |
|Ré         |11575      |
|Lá         |11255      |
|Ré Bemol   |10738      |
|Fá         |9321       |
|Si         |9235       |
|Mi         |8965       |
|Sol Bemol  |7885       |
|Si Bemol   |7414       |
|Lá Bemol   |7323       |
|Mi Bemol   |3542       |
+-----------+-----------+



In [109]:
# QUERY 3 - 20 maiores gêneros musicas com escala melódica menor
spark.sql('''SELECT genero, COUNT(genero) AS num_musicas
             FROM spotify
             WHERE esc_melodica = 'Menor'
             GROUP BY genero
             ORDER BY num_musicas DESC''').show(20,truncate=False)

+-----------------+-----------+
|genero           |num_musicas|
+-----------------+-----------+
|turkish          |668        |
|romance          |587        |
|deep-house       |538        |
|hardstyle        |531        |
|drum-and-bass    |526        |
|progressive-house|522        |
|tango            |520        |
|french           |516        |
|trance           |516        |
|hip-hop          |516        |
|trip-hop         |515        |
|dancehall        |511        |
|chicago-house    |501        |
|detroit-techno   |500        |
|study            |497        |
|house            |491        |
|afrobeat         |489        |
|reggaeton        |489        |
|electronic       |488        |
|anime            |487        |
+-----------------+-----------+
only showing top 20 rows



In [113]:
# QUERY 4 - Média de posivitidade da música de acordo com a escala melódica
spark.sql('''SELECT esc_melodica, AVG(positividade) AS med_positividade
             FROM spotify
             GROUP BY esc_melodica
             ORDER BY med_positividade DESC''').show(20,truncate=False)

+------------+-------------------+
|esc_melodica|med_positividade   |
+------------+-------------------+
|Maior       |0.47842858959904355|
|Menor       |0.46686528630482255|
+------------+-------------------+



In [124]:
# QUERY 4 - Quantidade de músicas explicitas e porcentagem
spark.sql('''SELECT cont_explic, COUNT(cont_explic) AS num_musicas
             FROM spotify
             GROUP BY cont_explic
             ORDER BY num_musicas DESC''').show(20,truncate=False)

+-----------+-----------+
|cont_explic|num_musicas|
+-----------+-----------+
|false      |103725     |
|true       |9697       |
+-----------+-----------+

